# Transfer Learning with MobileNetV3

<a name='1'></a>
## 1 - Packages

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## MobileNetV3


In [ ]:
IMG_SIZE = (512, 512)
IMG_SHAPE = IMG_SIZE + (3,)

mobile_v3_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(512,512,3),
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights=None,
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation='softmax',
    include_preprocessing=True
)

In [ ]:
print("Number of layers in the base model: ", len(mobile_v3_model.layers))

Number of layers in the base model:  263


In [ ]:
mobile_v3_model.summary()

Model: "MobilenetV3large"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 256, 256, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   

In [ ]:
[f'{layer.output_shape}  |  {layer.count_params()}  |  {layer.name}' for layer in mobile_v3_model.layers[-10: ]]

['(None, 16, 16, 960)  |  0  |  expanded_conv_14/squeeze_excite/Mul',
 '(None, 16, 16, 160)  |  153600  |  expanded_conv_14/project',
 '(None, 16, 16, 160)  |  640  |  expanded_conv_14/project/BatchNorm',
 '(None, 16, 16, 160)  |  0  |  expanded_conv_14/Add',
 '(None, 16, 16, 960)  |  153600  |  Conv_1',
 '(None, 16, 16, 960)  |  3840  |  Conv_1/BatchNorm',
 '(None, 16, 16, 960)  |  0  |  tf.__operators__.add_27',
 '(None, 16, 16, 960)  |  0  |  re_lu_38',
 '(None, 16, 16, 960)  |  0  |  tf.math.multiply_27',
 '(None, 16, 16, 960)  |  0  |  multiply_19']

# LVIS DataSet

In [1]:
!pip install tfds_nightly github-clone apache-beam
!rm -rf ../usr/local/lib/python3.7/dist-packages/tensorflow_datasets/object_detection/lvis
!ghclone https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/object_detection/lvis
!mv ./lvis ../usr/local/lib/python3.7/dist-packages/tensorflow_datasets/object_detection/

Cloning into 'lvis'...
done.


In [2]:
import tensorflow_datasets as tfds

In [ ]:
dataset, info = tfds.load(
    'lvis',
    split='train',
    download=True,
    shuffle_files=True,
    with_info=True,
    decoders=tfds.decode.PartialDecoding({
        'image': True,
        'features': tfds.features.FeaturesDict({'image/id':True,
                                                'objects':tfds.features.Sequence({
                                                    'id': True,
                                                    'bbox': True,
                                                    'label': tfds.features.ClassLabel(names=['skateboard','shoe'])
                                                })
        })
    })
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
tfds.show_examples(dataset, info)